# HW 3: Introduction and Setup
* Copy the contents of the file "transforms_hw03.py" to the bottom of your "transforms.py" file.
* Complete the function definitions for "se3" and "inv" 
* Run this notebook and check the outputs to make sure everything matches. If your code is correct, the output should exactly match the homogenous transformation matrices shown below. 
* Make sure to review each cell and think about what the operation means. Does it make sense? If not, ask about it on Piazza or in office hours. 

In [20]:
import transforms as tr
import numpy as np
from visualization import VizScene
import time

## Homogeneous matrix made of only rotation about z-axis by amount $\frac{\pi}{4}$

Rotation by amount theta around the z-axis should give the following:
$$
\left[\begin{matrix}0.707 & -0.707 & 0 & 0\\ 0.707 & 0.707 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [2]:
T = tr.se3(tr.rotz(np.pi/4))
T = np.round(T, 3)
print(T)


[[ 0.707 -0.707  0.     0.   ]
 [ 0.707  0.707  0.     0.   ]
 [ 0.     0.     1.     0.   ]
 [ 0.     0.     0.     1.   ]]


## Translation only along the x-axis by an amount $0.5$ should give the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5 \\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [3]:
T = tr.se3(p=[0.5, 0, 0])
print(T)

[[1.  0.  0.  0.5]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]


# Checking the "inv" function

If we calculate a homogenous transform with a rotation of 45 degrees about the x-axis and a general translation ($[0.5, 0.25, 0.3]^T$) we get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5\\0 & 0.707106781186548 & -0.707106781186547 & 0.25\\0 & 0.707106781186547 & 0.707106781186548 & 0.3\\0 & 0 & 0 & 1\end{matrix}\right]
$$

Then, we can multiply the original transform by its inverse to check out inverse function, we should get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1.0 & 0 & 0\\0 & 0 & 1.0 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [4]:
T = tr.se3(tr.rotx(np.pi/4), [0.5, 0.25, 0.3])

# round to 3 decimal places for better readability
T = np.round(T, 3)
print(T)

# now we can check if we implemented "inv" correctly:
check = tr.inv(T) @ T
check = np.round(check, 3)  # round to 3 decimal places for better readability
print("\n\n inv(T) @ T should give identity matrix:")
print(check)


[[ 1.     0.     0.     0.5  ]
 [ 0.     0.707 -0.707  0.25 ]
 [ 0.     0.707  0.707  0.3  ]
 [ 0.     0.     0.     1.   ]]


 inv(T) @ T should give identity matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


# Transform a point in one frame to another frame


## DH parameters combined for a single joint:

Remember that if we combine a rotation in z, translation in z, then translation in x, and rotation in x, we should get the same result as the book for following the DH convention to move from one frame (or joint) to another as follows: 

$$
\left[\begin{matrix}\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} \cos{\left(\alpha \right)} & \sin{\left(\alpha \right)} \sin{\left(\theta \right)} & a \cos{\left(\theta \right)}\\\sin{\left(\theta \right)} & \cos{\left(\alpha \right)} \cos{\left(\theta \right)} & - \sin{\left(\alpha \right)} \cos{\left(\theta \right)} & a \sin{\left(\theta \right)}\\0 & \sin{\left(\alpha \right)} & \cos{\left(\alpha \right)} & d\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In future homework, we'll implement representations of robot arms that include this transformation. But for the test values of DH parameters below, we can perform this sequence of operations manually.  

Assuming the following DH parameters for two joints:

$\theta_1 = \frac{\pi}{8}$, $d_1 = 0$, $a_1 = 0.3$, $\alpha_1 = \frac{\pi}{2}$

$\theta_2 = \frac{\pi}{4}$, $d_2 = 0$, $a_2 = 0.3$, $\alpha_2 = 0$


The resulting homogeneous transform describing the tip (or frame 2) relate to frame 0 would be:

$$
\left[\begin{matrix}0.653281482438188 & -0.653281482438188 & 0.38268343236509 & 0.473148304484842\\0.270598050073099 & -0.270598050073099 & -0.923879532511287 & 0.195984444731456\\0.707106781186548 & 0.707106781186548 & 0 & 0.212132034355964\\0 & 0 & 0 & 1.0\end{matrix}\right]
$$

In [24]:
# start by substituting the actual values for R and p and making a new SE3 object
# that describes the transformation from frame 0 to frame 1

# TODO - fill this out
# find the transformation from frame 0 to 1
T1_in_0 = tr.se3(R=tr.rotz(np.pi/8.0)) @tr.se3(p = np.array([0, 0, 0])) @tr.se3(p = np.array([0.3, 0, 0])) @tr.se3(R=tr.rotx(np.pi/2))

# do the same thing for frame 1 to frame 2
T2_in_1 = tr.se3(R=tr.rotz(np.pi/4.0)) @ tr.se3(p = np.array([0, 0, 0])) @ tr.se3(p = np.array([0.3, 0, 0])) @ tr.se3(R=tr.rotx(0.0))

# now we can combine the two to get a transformation that describes frame 2
# relative to frame 0
T2_in_0 = T1_in_0 @ T2_in_1

# round to 3 decimal places for better readability
T2_in_0 = np.round(T2_in_0, 3)

# printing the result
print(T2_in_0)

# use the "add_frame" function to plot both frames (for joint 1 and joint 2) relative
# to a base or ground frame.

# TODO - put your visualization code here.
viz = VizScene()
viz.add_frame(np.eye(4), label="ground")
viz.add_frame(T1_in_0, label="T1 in 0")
viz.add_frame(T2_in_0, label="T2 in 0")

run_time = 10  # seconds
refresh_rate = 30  # Hz

for i in range(run_time * refresh_rate):
    viz.update()
    time.sleep(1/refresh_rate)
    
viz.close_viz()

[[ 0.653 -0.653  0.383  0.473]
 [ 0.271 -0.271 -0.924  0.196]
 [ 0.707  0.707  0.     0.212]
 [ 0.     0.     0.     1.   ]]


## **2B**


In [27]:
T1_in_0 = tr.se3(p=[0, 1, 1])
T2_in_0 = tr.se3(p=[-0.5, 1.5, 1.1])
T3_in_0 = tr.se3(R=np.array([[0, 1, 0],[1, 0, 0],[0, 0, -1]]), p=[-0.5, 1.5, 3])

T3_in_2 = tr.inv(T2_in_0) @ T3_in_0

print(np.round(T1_in_0, 3))
print(np.round(T2_in_0, 3))
print(np.round(T3_in_0, 3))
print(np.round(T3_in_2, 3))

viz = VizScene()
viz.add_frame(np.eye(4), label="ground")
viz.add_frame(T1_in_0, label="T1 in 0")
viz.add_frame(T2_in_0, label="T2 in 0")
viz.add_frame(T3_in_0, label="T3 in 0")

viz.hold()


[[1. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
[[ 1.   0.   0.  -0.5]
 [ 0.   1.   0.   1.5]
 [ 0.   0.   1.   1.1]
 [ 0.   0.   0.   1. ]]
[[ 0.   1.   0.  -0.5]
 [ 1.   0.   0.   1.5]
 [ 0.   0.  -1.   3. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   1.   0.   0. ]
 [ 1.   0.   0.   0. ]
 [ 0.   0.  -1.   1.9]
 [ 0.   0.   0.   1. ]]


KeyboardInterrupt: 

## **2C** ##

In [ ]:
T_c = tr.se3(tr.rotz(np.pi/2.0))
T3_in_0_new = T3_in_0 @ T_c

T3_in_2 = tr.inv(T2_in_0) @ T3_in_0_new

print(np.round(T3_in_0_new, 3))
print(np.round(T3_in_2, 3))

[[ 1.   0.   0.  -0.5]
 [ 0.  -1.   0.   1.5]
 [ 0.   0.  -1.   3. ]
 [ 0.   0.   0.   1. ]]
[[ 1.   0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -1.   1.9]
 [ 0.   0.   0.   1. ]]


: 